## 1. 데이터 수집
네이버 뉴스 기사 사회면 헤드라인 웹크롤링 <br/>
출처: https://news.naver.com/

In [4]:
# 라이브러리 불러오기
from bs4 import BeautifulSoup 
import requests
from tqdm import tqdm
import datetime
import pandas as pd

In [5]:
def num():
    print("##########################################")
    num = int(input("원하는 숫자를 입력해주세요."))
    print("##########################################")
    return num

def news_crawling():
    ######### 네이버 url 가져올때, hearder을 안주는 경우 오류 발생시 ############
    headers = {
    "User-Agent":
    "Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) \
    Chrome/44.0.2403.157 Safari/537.36"
    }
    print("############# 네이버 뉴스 헤드라인 웹 크롤링 시작 ############")
    ### 네이버 뉴스 메인 페이지에서 상단바에서 뉴스 토픽을 설정해서 해당 토픽의 'a' 태그로 이동
    r = requests.get('https://news.naver.com/', headers=headers)
    soup = BeautifulSoup(r.text, 'html.parser')
    for title in range(len(soup.find("ul",'Nlnb_menu_list').find_all('li')[1:-4])):
        print(title+1 ,'', soup.find("ul",'Nlnb_menu_list').find_all('li')[1:-4][title].text)
    title_num = num() # title 선택
    title = soup.find("ul",'Nlnb_menu_list').find_all('li')[title_num].text # 나중에 데이터 저장하기 위해서 
    url = soup.find("ul",'Nlnb_menu_list').find_all('li')[title_num].find('a')['href']
    
    ### 이동된 세부 뉴스 토픽 페이지에서 왼쪽 사이드 바에서 세부 토픽 선택 후 해당 'a' 태그로 이동
    r = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.text, 'html.parser')
    for topic in range(len(soup.find('div',id='snb').find_all('li'))):
        print(topic+1 ,'', soup.find('div',id='snb').find_all('li')[topic].text)    
    topic_num = num()
    
    # a 태그로 이동하지 않고 / 한글을 입력하면 전체 뉴스 기사가 나옴
    a = int(input("요약본 1번 / 전체본 2번 선택해주세요."))
    if a == 1:
        minor_topic = soup.find('div',id='snb').find_all('li')[topic_num-1].find('a')['href'].split('&')[-1]
    else:
        minor_topic = soup.find('div',id='snb').find_all('li')[topic_num-1].text
    minor_topic1 = soup.find('div',id='snb').find_all('li')[topic_num-1].text # 나중에 데이터 저장할때 쓰기위함
    
    news_list = []
    date_list = []
    # 중간에 url이 main에서 list로 바껴야함
    news_url = url+'&'+minor_topic
    news_url = news_url.split('main.')[0]+'list.'+news_url.split('main.')[1]
    news_url = news_url.replace('LSD','LS2D') 
    print(news_url)
    
    ###################### 날짜 입력하는 부분은 안배움 ############################
    ###################### 밑 날짜 데이터 다루기 예시 참조 ########################
    print("##########################################")
    print('뉴스 크롤링을 시작할 날짜를 입력해주세요.')
    start = input("예: 20200101")
    print('뉴스 크롤링을 끝낼 날짜를 입력해주세요.')
    end = input("예: 20200101")
    print("##########################################")
    start = datetime.datetime.strptime(start, "%Y%m%d")
    end = datetime.datetime.strptime(end, "%Y%m%d")
    period = end-start
    date_generated = pd.date_range(start,periods=period.days)
    ###############################################################################
    
    # 날짜 별로 이동하기 위한 for문
    for date in range(len(date_generated)):
        r = requests.get(news_url+'&date='+str(date_generated[date].strftime("%Y%m%d")), headers=headers)
        print(news_url+'&date='+str(date_generated[date].strftime("%Y%m%d")))
        soup = BeautifulSoup(r.text, 'html.parser')
        # 끝 페이지를 알기위해서 임의의 높은 숫자를 입력하면 맨 끝 페이지로 이동 [실제 10000번째가 있으면 끝 아닐 수 있음]
        s = requests.get(news_url+'&date='+str(date_generated[date].strftime("%Y%m%d"))+
                         '&page=100000', headers=headers)
        s_soup = BeautifulSoup(s.text, 'html.parser')
        page_num = int(s_soup.find('div','paging').find('strong').text)
        ###### tqdm()을 for문에 사용하면 for문이 돌아가는 것을 시각화 할 수 있음.
        ## 페이지를 넘기는 부분
        for j in tqdm(range(page_num),desc=date_generated[date].strftime("%Y%m%d")):
            r = requests.get(news_url+'&date='+str(date_generated[date].strftime("%Y%m%d")+
                             '&page='+str(j+1)), headers=headers)
            soup = BeautifulSoup(r.text, 'html.parser')
            # 각 페이지 마다 기사를 가져오기 위한 부분
            for i in range(len(soup.find('div','list_body newsflash_body').find_all('li'))):
                date_list.append(date_generated[date].strftime("%Y%m%d"))
                news_list.append(soup.find('div','list_body newsflash_body').find_all('li')[i].find_all('a')[-1].text.strip())
    
    # 데이터 프레임으로 저장
    df = pd.DataFrame({"날짜":date_list,"헤드라인":news_list})
    
    # 데이터 프레임 이름 설정에 / 가 있으면 경로 오류가 생김으로 변경
    if '/' in minor_topic1 or '/' in title:
        title = title.replace('/','')
        minor_topic1 = minor_topic1.replace('/','')
        
    # 데이터 저장하기 [경로 맞게 지정해주기]
    df.to_csv('dataset/네이버뉴스_'+title+'_'+minor_topic1+'.csv',encoding='utf-8-sig')
    return df

In [6]:
df = news_crawling()

############# 네이버 뉴스 헤드라인 웹 크롤링 시작 ############
1  정치
2  경제
3  사회
4  생활/문화
5  IT/과학
6  세계
7  랭킹
##########################################
원하는 숫자를 입력해주세요.3
##########################################
1  사건사고 
2  교육 
3  노동 
4  언론 
5  환경 
6  인권/복지 
7  식품/의료 
8  지역 
9  인물 
10  사회 일반 
##########################################
원하는 숫자를 입력해주세요.6
##########################################
요약본 1번 / 전체본 2번 선택해주세요.1
https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid1=102&sid2=59b
##########################################
뉴스 크롤링을 시작할 날짜를 입력해주세요.
예: 2020010120220320
뉴스 크롤링을 끝낼 날짜를 입력해주세요.
예: 2020010120220325
##########################################
https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid1=102&sid2=59b&date=20220320


20220321:   0%|                                                                                  | 0/9 [00:00<?, ?it/s]

https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid1=102&sid2=59b&date=20220321


20220322:   0%|                                                                                  | 0/8 [00:00<?, ?it/s]

https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid1=102&sid2=59b&date=20220322


20220323:   0%|                                                                                 | 0/10 [00:00<?, ?it/s]

https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid1=102&sid2=59b&date=20220323


20220323: 100%|████████████████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  6.37it/s]


https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid1=102&sid2=59b&date=20220324


20220324: 100%|██████████████████████████████████████████████████████████████████████████| 9/9 [00:01<00:00,  6.19it/s]


In [ ]:
df

## 날짜 데이터 다루기 예시

In [7]:
# 문자열 20200111 을 날짜 형식에 맞춰서 년도/월/일로 변환
start = datetime.datetime.strptime('20200111', "%Y%m%d")
print(start)

2020-01-11 00:00:00


In [8]:
# start변수에는 날짜 타입
type(start)

datetime.datetime

In [9]:
end = datetime.datetime.strptime('20200211', "%Y%m%d")
print(end)

2020-02-11 00:00:00


In [10]:
# 날짜 끼리 연산
end - start

datetime.timedelta(days=31)

In [11]:
# 날짜 연산에서 빼진 날만 구하기
(end - start).days

31

In [12]:
# 해당 날짜 기간동안 날짜 데이터 레인지 생성
date_generated = pd.date_range(start, periods= (end - start).days)
date_generated

DatetimeIndex(['2020-01-11', '2020-01-12', '2020-01-13', '2020-01-14',
               '2020-01-15', '2020-01-16', '2020-01-17', '2020-01-18',
               '2020-01-19', '2020-01-20', '2020-01-21', '2020-01-22',
               '2020-01-23', '2020-01-24', '2020-01-25', '2020-01-26',
               '2020-01-27', '2020-01-28', '2020-01-29', '2020-01-30',
               '2020-01-31', '2020-02-01', '2020-02-02', '2020-02-03',
               '2020-02-04', '2020-02-05', '2020-02-06', '2020-02-07',
               '2020-02-08', '2020-02-09', '2020-02-10'],
              dtype='datetime64[ns]', freq='D')